## Full parameter search for new LPDC codes

#### Imports

In [1]:
import sys
import io
import numpy as np
import re
import time
import itertools
from numpy.linalg import matrix_power as matrix_power
from numpy.linalg import matrix_rank as matrix_rank


from mip import Model, xsum, minimize, BINARY
from bposd.css import css_code
from concurrent.futures import ThreadPoolExecutor
import multiprocessing

from parallel_functions import process_combination

#### Helper Functions

In [2]:
def get_net_encoding_rate(k, n):
    return k / (2*n)

In [3]:
# computes the minimum Hamming weight of a binary vector x such that
# stab @ x = 0 mod 2
# logicOp @ x = 1 mod 2
# here stab is a binary matrix and logicOp is a binary vector
def distance_test(stab,logicOp):
  # number of qubits
  n = stab.shape[1]
  # number of stabilizers
  m = stab.shape[0]

  # maximum stabilizer weight
  wstab = np.max([np.sum(stab[i,:]) for i in range(m)])
  # weight of the logical operator
  wlog = np.count_nonzero(logicOp)
  # how many slack variables are needed to express orthogonality constraints modulo two
  num_anc_stab = int(np.ceil(np.log2(wstab)))
  num_anc_logical = int(np.ceil(np.log2(wlog)))
  # total number of variables
  num_var = n + m*num_anc_stab + num_anc_logical

  model = Model()
  model.verbose = 0
  x = [model.add_var(var_type=BINARY) for i in range(num_var)]
  model.objective = minimize(xsum(x[i] for i in range(n)))

  # orthogonality to rows of stab constraints
  for row in range(m):
    weight = [0]*num_var
    supp = np.nonzero(stab[row,:])[0]
    for q in supp:
      weight[q] = 1
    cnt = 1
    for q in range(num_anc_stab):
      weight[n + row*num_anc_stab +q] = -(1<<cnt)
      cnt+=1
    model+= xsum(weight[i] * x[i] for i in range(num_var)) == 0

  # odd overlap with logicOp constraint
  supp = np.nonzero(logicOp)[0]
  weight = [0]*num_var
  for q in supp:
    weight[q] = 1
  cnt = 1
  for q in range(num_anc_logical):
      weight[n + m*num_anc_stab +q] = -(1<<cnt)
      cnt+=1
  model+= xsum(weight[i] * x[i] for i in range(num_var)) == 1

  model.optimize()

  opt_val = sum([x[i].x for i in range(n)])

  return int(opt_val)


def distance_test_parallel(stab, logicOp):
    n = stab.shape[1]  # number of qubits
    m = stab.shape[0]  # number of stabilizers

    # Parallel computation of wstab
    with ThreadPoolExecutor() as executor:
        wstab_list = list(executor.map(np.sum, stab))
    wstab = np.max(wstab_list)

    wlog = np.count_nonzero(logicOp)
    num_anc_stab = int(np.ceil(np.log2(wstab)))
    num_anc_logical = int(np.ceil(np.log2(wlog)))
    num_var = n + m * num_anc_stab + num_anc_logical

    model = Model()
    model.verbose = 0
    x = [model.add_var(var_type=BINARY) for i in range(num_var)]
    model.objective = minimize(xsum(x[i] for i in range(n)))

    # Function to prepare and add a constraint for a row of stab
    def add_stab_constraint(row):
        weight = [0] * num_var
        supp = np.nonzero(stab[row, :])[0]
        for q in supp:
            weight[q] = 1
        cnt = 1
        for q in range(num_anc_stab):
            weight[n + row * num_anc_stab + q] = -(1 << cnt)
            cnt += 1
        return xsum(weight[i] * x[i] for i in range(num_var)) == 0

    # Parallel addition of orthogonality constraints
    with ThreadPoolExecutor() as executor:
        constraints = list(executor.map(add_stab_constraint, range(m)))
    for constraint in constraints:
        model += constraint

    # Adding odd overlap with logicOp constraint (not parallelized)
    supp = np.nonzero(logicOp)[0]
    weight = [0] * num_var
    for q in supp:
        weight[q] = 1
    cnt = 1
    for q in range(num_anc_logical):
        weight[n + m * num_anc_stab + q] = -(1 << cnt)
        cnt += 1
    model += xsum(weight[i] * x[i] for i in range(num_var)) == 1

    model.optimize()
    opt_val = sum(x[i].x for i in range(n))

    return int(opt_val)

In [4]:
def search_codes(max_ell, max_m, num_summands_a, num_summands_b):
    good_configs = []

    for ell in range(6, max_ell + 1):
        for m in range(6, max_m + 1):
            I_ell = np.identity(ell, dtype=int)
            I_m = np.identity(m, dtype=int)
            x, y = {}, {}
            for i in range(ell):
                x[i] = np.kron(np.roll(I_ell, i, axis=1), I_m)
            for j in range(m):
                y[j] = np.kron(I_ell, np.roll(I_m, j, axis=1))

            # Iterate over all possible fixed indices for x in A and y in B
            for fixed_x_index_a in range(1, 4):
                for fixed_y_index_b in range(1, 4):
                    # Exclude the fixed indices from the possible combinations
                    x_indices = [i for i in range(ell) if i != fixed_x_index_a]
                    y_indices = [i for i in range(m) if i != fixed_y_index_b]

                    # Iterate over combinations for the non-fixed terms
                    for combo_a in itertools.combinations(y_indices, num_summands_a - 1):  # -1 for the fixed x term
                        for combo_b in itertools.combinations(x_indices, num_summands_b - 1):  # -1 for the fixed y term
                            # Construct A and B with the fixed term and current combinations
                            A = x[fixed_x_index_a]
                            for idx in combo_a:
                                A += y[idx]
                            A %= 2

                            B = y[fixed_y_index_b]
                            for idx in combo_b:
                                B += x[idx]
                            B %= 2

                            # Construct polynomial sum strings for A and B
                            A_poly_sum = f"x{fixed_x_index_a}" + " + " + " + ".join(f"y{idx}" for idx in combo_a)
                            B_poly_sum = f"y{fixed_y_index_b}" + " + " + " + ".join(f"x{idx}" for idx in combo_b)

                            # Transpose matrices A and B
                            AT = np.transpose(A)
                            BT = np.transpose(B)

                            # Construct matrices hx and hz
                            hx = np.hstack((A, B))
                            hz = np.hstack((BT, AT))

                            # Construct the CSS code and test it
                            qcode = css_code(hx, hz)  # Assuming css_code is defined elsewhere
                            if qcode.test():  # Assuming the test method is defined for qcode
                                r = get_net_encoding_rate(qcode.K, qcode.N)  # Assuming this function is defined
                                if r > 1/15:  # Your specific criteria for good configurations
                                    code_config = {
                                        'ell': ell,
                                        'm': m,
                                        'n_phys_qubits': qcode.N,
                                        'n_log_qubits': qcode.K,
                                        'encoding_rate': r,
                                        'A_poly_sum': A_poly_sum,
                                        'B_poly_sum': B_poly_sum
                                    }
                                    good_configs.append(code_config)

    return good_configs


In [5]:
### Surpress the print statements of the qcode.test() function
# Save the current stdout so we can restore it later
original_stdout = sys.stdout
# Redirect stdout to a dummy StringIO object
sys.stdout = io.StringIO()

# Example usage
good_configs = search_codes(max_ell=6, max_m=6, num_summands_a=3, num_summands_b=3)
len(good_configs)

4

#### Verify if the code in the paper can be found/reproduced with the code search function

In [6]:
# Look for the first code in the paper of IBM
criteria = {'ell': 6, 'm': 6, 'n_phys_qubits': 72, 'n_log_qubits': 12}

filtered_list = [d for d in good_configs if all(d.get(key) == value for key, value in criteria.items())]

def find_matching_config(configs, a_poly, b_poly):
    for config in configs:
        if config.get('A_poly_sum') == a_poly and config.get('B_poly_sum') == b_poly:
            print('Found code in the paper!')
            return config  # Return the matching config if found
    raise ValueError('Code in the paper could not be found! Verify the code search function!')  # Return None if no match is found

paper_config = find_matching_config(good_configs, 'x3 + y1 + y2', 'y3 + x1 + x2')
paper_config

{'ell': 6,
 'm': 6,
 'n_phys_qubits': 72,
 'n_log_qubits': 12,
 'encoding_rate': 0.08333333333333333,
 'A_poly_sum': 'x3 + y1 + y2',
 'B_poly_sum': 'y3 + x1 + x2'}

### Parallelization of code search

In [7]:
def build_code(max_ell, max_m, num_summands_a, num_summands_b):
    all_configs = []
    args_list = []

    for ell in range(6, max_ell + 1):
        for m in range(6, max_m + 1):
            for fixed_x_exponent_a in range(1, 4):
                for fixed_y_exponent_b in range(1, 4):
                    args_list.append((ell, m, fixed_x_exponent_a, fixed_y_exponent_b, num_summands_a, num_summands_b))

    # Determine the number of processes to use
    num_processes = min(multiprocessing.cpu_count(), len(args_list))

    with multiprocessing.Pool(processes=num_processes) as pool:
        results = pool.map(process_combination, args_list)

    # Flatten the list of lists
    for result in results:
        all_configs.extend(result)

    return all_configs

In [8]:
### Surpress the print statements of the qcode.test() function
# Save the current stdout so we can restore it later
original_stdout = sys.stdout
# Redirect stdout to a dummy StringIO object
sys.stdout = io.StringIO()

good_configs_parallel = build_code(
    max_ell=6, 
    max_m=6, 
    num_summands_a=3, 
    num_summands_b=3
)

In [9]:
len(good_configs_parallel)

4

In [10]:
filtered_list = [d for d in good_configs_parallel if all(d.get(key) == value for key, value in criteria.items())]

def find_matching_config(configs, a_poly, b_poly):
    for config in configs:
        if config.get('A_poly_sum') == a_poly and config.get('B_poly_sum') == b_poly:
            print('Found code in the paper!')
            return config  # Return the matching config if found
    raise ValueError('Code in the paper could not be found! Verify the code search function!')  # Return None if no match is found

paper_config = find_matching_config(good_configs_parallel, 'x3 + y1 + y2', 'y3 + x1 + x2')
paper_config

{'ell': 6,
 'm': 6,
 'n_phys_qubits': 72,
 'n_log_qubits': 12,
 'encoding_rate': 0.08333333333333333,
 'A_poly_sum': 'x3 + y1 + y2',
 'B_poly_sum': 'y3 + x1 + x2'}

#### Rebuild a code from a configuration

In [11]:
import numpy as np
import re  # For parsing the polynomial strings

def rebuild_code(config):
    # Extract configuration details
    ell = config['ell']
    m = config['m']
    A_poly_sum = config['A_poly_sum']
    B_poly_sum = config['B_poly_sum']

    # Define cyclic shift matrices
    I_ell = np.identity(ell, dtype=int)
    I_m = np.identity(m, dtype=int)
    x, y = {}, {}
    for i in range(ell):
        x[i] = np.kron(np.roll(I_ell, i, axis=1), I_m)
    for i in range(m):
        y[i] = np.kron(I_ell, np.roll(I_m, i, axis=1))

    # Initialize A and B matrices
    A = np.zeros((ell*m, ell*m), dtype=int)
    B = np.zeros((ell*m, ell*m), dtype=int)

    # Parse A_poly_sum and B_poly_sum to construct A and B
    for term in re.findall(r'([xy]\d+)', A_poly_sum):
        idx = int(term[1:]) - 1  # Convert term to 0-based index
        if term.startswith('x') and idx < ell:
            A += x[idx]
        elif term.startswith('y') and (idx < m):
            A += y[idx]

    for term in re.findall(r'([xy]\d+)', B_poly_sum):
        idx = int(term[1:]) - 1  # Convert term to 0-based index
        if term.startswith('x') and idx < ell:
            B += x[idx]
        elif term.startswith('y') and (idx < m):
            B += y[idx]

    # Ensure matrices are binary
    A %= 2
    B %= 2

    # Transpose matrices A and B
    AT = np.transpose(A)
    BT = np.transpose(B)

    # Construct matrices hx and hz
    hx = np.hstack((A, B))
    hz = np.hstack((BT, AT))

    qcode = css_code(hx, hz)

    # Construct and return the CSS code using hx and hz
    return {
        'qcode': qcode,
        'hx': hx,
        'hz': hz,
    }

In [12]:
code_info = rebuild_code(paper_config)
print('CSS code rebuilt based on the configuration.')

In [13]:
qcode = code_info.get('qcode')

hx, hz = code_info.get('hx'), code_info.get('hz')
n, k = qcode.N, qcode.K
lx, lz =  qcode.lx, qcode.lz

In [ ]:
start_time = time.time()

print('Computing code distance PARALLELIZED...')
# We compute the distance only for Z-type logical operators (the distance for X-type logical operators is the same)
# by solving an integer linear program (ILP). The ILP looks for a minimum weight Pauli Z-type operator which has an even overlap with each X-check
# and an odd overlap with logical-X operator on the i-th logical qubit. Let w_i be the optimal value of this ILP.
# Then the code distance for Z-type logical operators is dZ = min(w_1,…,w_k).
d = qcode.get('n')
for i in range(qcode.get('k')):
	w = distance_test_parallel(hx, lx[i,:])
	print('Logical qubit =', i+1, 'Distance =', w)
	d = min(d, w)

print(f'\nCode parameters: [n, k, d] = [{n}, {k}, {d}]')

print('Execution Time: ', round(time.time() - start_time, 1), 'seconds')